In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import sys

import sys
sys.path.append("/Workspace/Users/mandu543@gmail.com/databricks-movies-analytics/Movies_Project")

# Maintenant on peut importer
from src.utils import *

# Liste tous les schemas du catalog 'workspace'
spark.sql("SHOW SCHEMAS IN workspace").show()

print(VOLUME_PATH)
print(BRONZE_TABLE)
print(CSV_EXTENSION)


### Ingestion des données


In [0]:
#RQ : Pout utiliser : %run  faut transformer les fichiers config et transformation en notebook
#%run /Users/mandu543@gmail.com/Movies_Project/src/config
#%run /Workspace/Users/mandu543@gmail.com/Movies_Project/scr/transformations

Récupération des fichiers et insertion dans le couche bronze (tmdb_movies)

In [0]:
# -----------------------------
# Lister tous les fichiers CSV du Volume
# -----------------------------
csv_files = [f.path for f in dbutils.fs.ls(VOLUME_PATH) if f.name.endswith(CSV_EXTENSION)]
print(f"Fichiers CSV trouvés : {csv_files}")

# -----------------------------
# Lire tous les CSV avec options sécurisées
# -----------------------------
# Lire CSV avec première ligne comme header
# Gérer les champs multi-lignes, guillemets, virgules et espaces inutiles
dfs = []
for file in csv_files:
    df = spark.read \
            .option("header", True) \
            .option("multiLine", True) \
            .option("escape", '"') \
            .option("quote", '"') \
            .option("ignoreLeadingWhiteSpace", True) \
            .csv(file)
dfs.append(df)
# -----------------------------
# Combiner tous les CSV
# -----------------------------
df_bronze = dfs[0]
for df in dfs[1:]:
    df_bronze = df_bronze.unionByName(df)
# Create schema if not exists
spark.sql(
    f"CREATE SCHEMA IF NOT EXISTS {BRONZE_TABLE.split('.')[1]}"
)
# Écriture Delta Bronze
df_bronze.write.format("delta").mode("overwrite").saveAsTable(BRONZE_TABLE)

print(f"✅ Bronze table created with {df_bronze.count()} rows")


### Voir les versions des insertions grâce au format delta

Meme si overwrite fait un truncate, le format delta peut récupérer des versions et restaurer les versions souhaitées

In [0]:
# Voir versions
spark.sql("DESCRIBE HISTORY "+BRONZE_TABLE+"").show()

In [0]:
# Lire version précédente
#df_prev = spark.read.format("delta").option("versionAsOf", 0).table("workspace.bronze.tmdb_movies")

# Restaurer table
#df_prev.write.format("delta").mode("overwrite").saveAsTable("workspace.bronze.tmdb_movies")